# Analyzing the Data
- **Name: Andrew Angulo**
- **Data Systems Project**

The purpose of Analyzing the data is to visualize and represent the information we have stored within SQL, doing this further allows us to answer our central question: **are some characters are better than others based on their stats, and how much would the player have to invest in the game to acquire that character?**

---
### Description of Imports:

- we use **pandas** to represent our data from SQL into a tabular dataframe


- we use **os.path** in order to obtain the file based on the location specificed within the computer


- we use **json** so we are able to load data from a JSON files such as creds.json


- we use **sqlalchemy** so we are able to interact with the SQL database

In [1]:
import pandas as pd
import os.path
import json
import sqlalchemy as sa


%load_ext sql

---
### Utilizing the functions
Below are functions that help us retrieve information in order to help us interact with the SQL database
- **getsqlite_creds** : strings
    - returns strings of the specificed content within creds.json
    
    
- **buildConnectionString** : string
    - returns a string with the specified path of the database file based on the previouse function

In [2]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [3]:
def buildConnectionString(source):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

### Connecting to the SQL Database

With all of the functions utilized above we are finally allowed to
connect to the SQL database as if everything passed we connect to the engine

In [4]:
# Build the conection string
cstring = buildConnectionString("sqlite_arknights")
print("Connection string:", cstring)

# Connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

Connection string: sqlite:///dbfile/arknights.db


In [5]:
%sql $cstring

### How is the data stored?
Currently all of the data we are retrieving from the SQL database is stored in a **.db file** within the folder **dbfile**. Entering this file presents us with all of the tables that it contains. The tables within this database files also contains constraints which allows certain values to have a certain condition set to them

### What Queries will I create?
The queries I will create will be the following

- **Query 1:**
    - The first query will inner join operators table and banners table so it can present all of the characters stats of that upcoming banner
    
- **Query 2:**
    - The second query will also do an inner join on operators table and popular table so it can show all of the stats  of the current popular characters
    
- **Query 3:**
    - The third and final query will do an inner join on popular, banners, and operators so that it presents any of the popular characters that possibly come within the upcoming banners with their stats.

---
### Query1

Now that we are connected to the SQL Database we are able to perform querys.
Below is a query where it inner joins operators table and the banner table using characterID as the primary key. Looking at this SQL database allows us to visually think about if an upcoming character within the specific banner is worth to pull for based on their stats.

In [10]:
query1 = """
SELECT *
FROM operators AS op INNER JOIN banners AS ban USING(characterID)
"""


resultset1 = %sql $query1
resultdf1 = resultset1.DataFrame()

resultdf1

 * sqlite:///dbfile/arknights.db
Done.


,characterID,HP,ATK,DEF,COST,RES,BLOCK,REDEPLOY,INTERVAL,TARGET,DMG,ROLE,bannerName,ChineseDate
0,Blue Poison,1230,610,130,13,5,1,70,1.00,1,Physical,Sniper,Dissonanzen,2022-06-09 - 2022-06-23
1,Ptilopsis,1610,390,150,17,0,1,70,2.85,3,Healing,Medic,Pathfinder of Sands,2022-07-05 - 2022-07-19
2,Gavial the Invincible,2906,816,451,24,0,3,70,1.20,AoE,Physical,Guard,Great Axe and Pen Nib - [Summer] Series Limite...,2022-08-11 - 2022-08-25
3,Skyfire,1620,874,122,33,20,1,70,2.90,AoE,Arts,Caster,Joint Operation 7,2022-08-25 - 2022-09-08
4,Franka,3768,1011,260,18,0,1,70,1.50,1,Physical,Guard,Never Vowed,2022-09-08 - 2022-09-22
5,Jackie,2378,589,308,9,0,1,70,0.78,1,Physical,Guard,The Front That Was,2022-09-27 - 2022-10-11
6,Cliffheart,1970,835,340,13,0,2,70,1.80,1,Physical,Specialist,Bearing and Sparks,2022-10-11 - 2022-10-25
7,Texas the Omertosa,1598,659,320,10,0,1,18,0.93,1,Physical,Specialist,Chop the Thorns: Open Circuits - Celebration S...,2022-11-01 - 2022-11-15
8,Sora,1356,385,258,7,0,1,70,1.00,1,Healing,Supporter,Joint Operation 8,2022-11-15 - 2022-11-29
9,Pramanix,1605,495,102,12,25,1,70,1.60,1,Arts,Supporter,Edelweiss' Vow Rerun,2022-12-01 - 2022-12-15


### Query2

Query 2 uses INNER JOIN within operators table and the popular table, thus results in returning a pandas datframe translated from SQL. The goal of this query is that it shows the top characters that are popular in the present day but at the same time it also presents the stats of that specific character. This helps us answer our central question because this allows the player to further assume if other characters are better than other characters they possibly had in mind

In [7]:
query2 = """
SELECT *
FROM operators AS op INNER JOIN popular AS pop USING(characterID)
ORDER BY ATK DESC
"""


resultset2 = %sql $query2
resultdf2 = resultset2.DataFrame()

resultdf2

 * sqlite:///dbfile/arknights.db
Done.


,characterID,HP,ATK,DEF,COST,RES,BLOCK,REDEPLOY,INTERVAL,TARGET,DMG,ROLE
0,Erato,1798,1027,125,23,0,1,70,2.40,1,Physical,Sniper
1,Mudrock,4428,882,662,36,10,3,70,1.60,1,Physical,Defender
2,Gladiia,2309,851,381,16,0,2,80,1.80,1,Physical,Specialist
3,Specter the Unchained,2803,817,322,16,0,2,70,1.20,1,Physical,Specialist
4,Gavial the Invincible,2906,816,451,24,0,3,70,1.20,AoE,Physical,Guard
5,Irene,2935,701,369,23,0,2,70,1.30,1,Physical,Guard
6,Texas the Omertosa,1598,659,320,10,0,1,18,0.93,1,Physical,Specialist
7,Exusiai,1673,630,161,14,0,1,70,1.00,1,Physical,Sniper
8,Lumen,1825,585,141,23,10,1,80,2.85,1,Healing,Medic
9,Kal'tsit,2033,490,255,20,0,1,70,2.85,1,Healing,Medic


### Query3

Query 3 uses an INNER JOIN within the popular table and the banners table. The goal of this is to represent if any of the popular characters users are interested today are going to be available within the upcoming banners, this will allow users to question if they should save their resources for this upcoming banner to pull the desired operator.

In [8]:
query3 = """
SELECT *
FROM popular AS pop INNER JOIN  banners AS ban USING(characterID) INNER JOIN operators as op USING(characterID)
"""


resultset3 = %sql $query3
resultdf3 = resultset3.DataFrame()

resultdf3

 * sqlite:///dbfile/arknights.db
Done.


,characterID,bannerName,ChineseDate,HP,ATK,DEF,COST,RES,BLOCK,REDEPLOY,INTERVAL,TARGET,DMG,ROLE
0,Gavial the Invincible,Great Axe and Pen Nib - [Summer] Series Limite...,2022-08-11 - 2022-08-25,2906,816,451,24,0,3,70,1.20,AoE,Physical,Guard
1,Texas the Omertosa,Chop the Thorns: Open Circuits - Celebration S...,2022-11-01 - 2022-11-15,1598,659,320,10,0,1,18,0.93,1,Physical,Specialist
2,Mudrock,The Front That Was,2022-09-27 - 2022-10-11,4428,882,662,36,10,3,70,1.60,1,Physical,Defender
